In [30]:
import os
import numpy as np
import time
import pickle
import imp

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torchsummary import summary


from cruw import CRUW
from rodnet.datasets.CRDataset import CRDataset
from rodnet.datasets.collate_functions import cr_collate
from rodnet.utils.solve_dir import create_dir_for_new_model
from rodnet.utils.load_configs import load_configs_from_file, parse_cfgs, update_config_dict
from rodnet.utils.visualization import visualize_train_img, visualize_test_img_wo_gt, visualize_confmap, visualize_test_img
from rodnet.core.post_processing import ConfmapStack, post_process, post_process_single_frame, write_dets_results, write_dets_results_single_frame, lnms, ops

In [2]:
config_dict = load_configs_from_file('/Users/kushalgadhiya/Desktop/Winter2024/ENDG 511/Project/RODNet-master/configs/config_rodnet_cdc_win16.py')
dataset = CRUW(data_root=config_dict['dataset_cfg']['base_root'], sensor_config_name='sensor_config_rod2021')
radar_configs = dataset.sensor_cfg.radar_cfg
range_grid = dataset.range_grid         
angle_grid = dataset.angle_grid         
model_cfg = config_dict['model_cfg']

In [3]:
crdata_train = CRDataset(data_dir='/Volumes/T7MAC/data_v2', dataset=dataset, config_dict=config_dict, split='train',
                                 noise_channel= False)
crdata_test = CRDataset(data_dir='/Volumes/T7MAC/data_v2', dataset=dataset, config_dict=config_dict, split='test',
                                 noise_channel= False)

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00, 213.60it/s]


In [4]:
print(len(crdata_train))
print(len(crdata_test))

2416
325


In [27]:
import getModel
imp.reload(getModel)
from getModel import ROD_V0, ROD_V1
import trainModel
imp.reload(trainModel)
from trainModel import modelHandler,baseHandler

In [6]:
n_class = dataset.object_cfg.n_class          
n_epoch = config_dict['train_cfg']['n_epoch']   
batch_size = config_dict['train_cfg']['batch_size'] 
lr = config_dict['train_cfg']['lr']
train_configs = config_dict['train_cfg']
test_configs = config_dict['test_cfg']
win_size = train_configs['win_size']
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
tdataloader = DataLoader(crdata_train, batch_size, shuffle=True, num_workers=0, collate_fn=cr_collate)
vdataloader = DataLoader(crdata_test, batch_size=1,shuffle=True, num_workers=0, collate_fn=cr_collate)

In [18]:
train_model_path = '/Volumes/T7MAC/trainedrepo'         #entre your desired location to store trained model
if not os.path.exists(train_model_path):
    model_dir, model_name = create_dir_for_new_model(model_cfg['name'], train_model_path)

In [28]:
rod_v1 = ROD_V1(in_channels=2, n_class=n_class)

optimizer_v1 = optim.Adam(rod_v1.parameters(), lr=lr)
criterion_v1 = nn.BCELoss()
scheduler_v1 = StepLR(optimizer_v1, step_size=config_dict['train_cfg']['lr_step'], gamma=0.1)

M_v1 = modelHandler(rod_v1,criterion_v1,optimizer_v1,device,scheduler_v1,n_epoch)

Number of encoder parameters: 6481152
Number of decoder short parameters: 1200453
Number of decoder long parameters: 28039110


In [29]:
rodNet_v1, history_v1 = M_v1.train(tdataloader,model_dir)

0.7276224493980408
0.7187356948852539


KeyboardInterrupt: 